In [1]:
import pandas as pd
from pandas import Series, DataFrame
import math
%matplotlib inline
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

## 这里需要根据输入表格的列的名称去修改 （必须去确定pusle 列）

In [143]:
#读取原始数据 
test = pd.read_csv("../data/sensor_data_origin/220617_1755_ethanol5ppm.csv")
# 第八行无名称的数据改成pulse
test.rename(columns={'Unnamed: 8': 'pulse'},inplace=True)


## 从这里开始包装成函数调用

In [144]:
origin_data_length = 90
traget_data_length = 70
all = test
count = math.floor(len(test)/origin_data_length) # math.floor 向下取的整数 筛选的一个周期为90个数据
zero = [0]*count

## 这里可能需要有个数据接口 对应不同的脉冲电压 如 0.2-1.8的情况 （<span class="burk">列名称必须是pulse</span>）

In [145]:
#### 注意这里数据的列名称必须是 pusle
for j in range(count):
    
    for i in range (j*origin_data_length,(j+1)*origin_data_length):
        if test.loc[i,"pulse"] > 2.5 and test.loc[(i-1),"pulse"] < 0.5: #根据脉冲的特性找到跳变开始的点   
            zero[j] = test.index[i]

In [146]:
for i in zero:
    p = zero.index(i)
    #print(p)
    for j in range(traget_data_length):  # traget_data_length 是脉冲前20个点 脉冲过程30个  脉冲后20个
        all.iloc[p*traget_data_length+j,:] = test.iloc[i-20+j,:] # 这的20 代表往前取了20个点 （这个需不需要做成接口  待定）

In [147]:
all.to_csv('../data/process/allout.csv') 

In [148]:
data_set =pd.read_csv ('../data/process/allout.csv')

In [149]:
redundancy = all.shape[0]%traget_data_length

In [150]:
all_feature = all.iloc[0:-redundancy,1:] #读取整数个样本数的数据

In [151]:
all_feature.head()

,sensor1(vol),sensor2(vol),sensor3(vol),sensor4(vol),sensor5(vol),sensor6(vol),sensor7(vol),pulse
0,0.023811,0.021556,0.020589,0.023167,0.039600,0.041211,1.056189,0.446235
1,0.023811,0.021878,0.023167,0.023811,0.040889,0.042178,1.053933,0.445591
2,0.023489,0.020911,0.021878,0.022522,0.040889,0.042500,1.055222,0.446880
3,0.023489,0.021878,0.021556,0.023489,0.041211,0.040889,1.056833,0.446880
4,0.023811,0.020911,0.022845,0.022522,0.040566,0.042178,1.054900,0.445913


In [152]:
sensor1 = all_feature.iloc[:,0] # 四列数据对应四个传感器
sensor2 = all_feature.iloc[:,1]
sensor3 = all_feature.iloc[:,2]
sensor4 = all_feature.iloc[:,3]

In [153]:
# 把sensor的数据弄成 traget-data_length的宽度的数据
train_features = torch.Tensor(sensor1.values)
true_train_features1 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor2.values)
true_train_features2 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor3.values)
true_train_features3 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor4.values)
true_train_features4 = train_features.reshape(-1,traget_data_length)

## 还需要添加一个湿度的变量（）

In [154]:
#这里把直流的数据当成label  后续测试 改成两个直流的
label = all_feature.iloc[:,6]
train_label = torch.tensor(label.values)
true_train_label = train_label.reshape(-1,traget_data_length)

In [155]:
a = true_train_features1.detach().numpy()
b = true_train_features2.detach().numpy()
c = true_train_features3.detach().numpy()
d = true_train_features4.detach().numpy()
l = true_train_label.detach().numpy()

In [156]:
a1 = pd.DataFrame(a)
b1 = pd.DataFrame(b)
c1 = pd.DataFrame(c)
d1 = pd.DataFrame(d)
l1 = pd.DataFrame(l)

In [157]:
# 把label的值增加到数据中去  把第一个直流的值当成label
a1.loc[:,traget_data_length] = l1.iloc[:,0]
b1.loc[:,traget_data_length] = l1.iloc[:,0]
c1.loc[:,traget_data_length] = l1.iloc[:,0]
d1.loc[:,traget_data_length] = l1.iloc[:,0]

## 需要改变文件名称

In [158]:
a1.to_csv('../data/process/sensor1_5ppm.csv',index=0)
b1.to_csv('../data/process/sensor2_5ppm.csv',index=0)
c1.to_csv('../data/process/sensor3_5ppm.csv',index=0)
d1.to_csv('../data/process/sensor4_5ppm.csv',index=0)

# 不同浓度采样

In [166]:
read_data_1ppm = pd.read_csv('../data/process/sensor1_1ppm.csv')
read_data_5ppm = pd.read_csv('../data/process/sensor1_5ppm.csv')
read_data_10ppm = pd.read_csv('../data/process/sensor1_10ppm.csv')  #  需要修改读取的文件  index =0 列号是否选

In [167]:
train_data_1ppm = read_data_1ppm.iloc[2000:3000,0:-1]
label_data_1ppm = read_data_1ppm.iloc[2000:3000,-1]
train_data_5ppm = read_data_5ppm.iloc[2000:3000,0:-1]
label_data_5ppm = read_data_5ppm.iloc[2000:3000,-1]
train_data_10ppm = read_data_10ppm.iloc[2000:3000,0:-1]
label_data_10ppm = read_data_10ppm.iloc[2000:3000,-1]

In [168]:
# 数据归一化 每行都除以改行的最大值  
######不需要归一化 可以注释掉这里 ---------------------------########
df1 = train_data_1ppm.max(axis=1)
df2 = train_data_5ppm.max(axis=1)
df3 = train_data_10ppm.max(axis=1)
train_data_1ppm=train_data_1ppm.div(df1, axis='rows')
train_data_5ppm=train_data_5ppm.div(df2, axis='rows')
train_data_10ppm=train_data_10ppm.div(df3, axis='rows')

In [169]:
#添加label列  0 1 2 softmax特有的
train_data_1ppm.loc[:,71]=0
train_data_5ppm.loc[:,71]=1
train_data_10ppm.loc[:,71]=2

In [170]:
test_data_noppm = pd.concat([train_data_1ppm,train_data_5ppm,train_data_10ppm],axis=0)

In [171]:
#随机采样 生成学习数据
example = test_data_noppm.sample(1000,replace=False,random_state=42,axis=0)
train_set = example.iloc[0:800,:]
test_set = example.iloc[800:1000,:]
train_set.to_csv("../data/train_set.csv",index=0)
test_set.to_csv("../data/test_set.csv",index=0)

In [22]:
type(train_set)

pandas.core.frame.DataFrame

## 将产生的数据集做归一化

In [56]:
read_test_set = pd.read_csv('../../../data/history/softmax-0914/train_s2 for tets_s2/test_set.csv')
read_train_set = pd.read_csv('../../../data/history/softmax-0914/train_s2 for tets_s2/train_set.csv')


In [57]:
ab = read_test_set.iloc[:,0:-1]
ab1 = ab.max(axis=1)
ab = ab.div(ab1,axis='rows')
ab.loc[:,71] = read_test_set.iloc[:,-1]
ab.to_csv("../../../data/history/softmax-0914/train_s2 for tets_s2/test_set_normol.csv",index=0)

In [62]:
bc = read_train_set.iloc[:,0:-1]
bc1 = bc.max(axis=1)
bc = bc.div(bc1,axis='rows')
bc.loc[:,71] = read_train_set.iloc[:,-1]
bc.to_csv("../../../data/history/softmax-0914/train_s2 for tets_s2/train_set_normol.csv",index=0)